In [1]:
import sqlite3
import pandas as pd
import re
from dfply import *


In [249]:
con = sqlite3.connect("")

df = pd.read_sql_query("", con)
con.close

<function Connection.close()>

In [74]:
sample_num = 8001
print(df['sys_prompt'][sample_num])#sys_prompt, obs_prompt, chosen, rejected
print(df['obs_prompt'][sample_num])
print(df['chosen'][sample_num])
print(df['rejected'][sample_num])
print(df['game'][sample_num])
print(df.shape)

You are a powerful gaming agent who can make proper decisions to beat the user in gaming tasks. You are a helpful assistant that strictly follows the user's instructions.
You and your partner are in the Prisoner's Dilemma situation. Specifically, if you <Testify> against your partner and your partner remains <Silent>, you will go free while your partner will get 3 years in prison on the main charge. If you remain <Silent> but your partner <Testify> against you, you will serve 3 years in prison and your partner will be set free. If you and your partner <Testify> against each other, you and your partner will each serve 2 years. 
If both you and your partner remain <Silent>, you and your partner will each server 1 year.
You have been through some this situation in the past and here are the decisions you and your partner made:
In the 1th round, you decided to <Silent> and your opponent decided to <Silent>.
In the 2th round, you decided to <Silent> and your opponent decided to <Silent>.

In

In [250]:
df2 = df.drop_duplicates()
df2 >> group_by(X.game) >> summarize(total =X.game.count()) 

,game,total
0,breakthrough,20278
1,connect4,29940
2,first_sealed_auction,53
3,kuhn_poker,17
4,liars_dice,140
5,negotiation,1056
6,nim,560
7,pig,444
8,python_iterated_prisoners_dilemma,662
9,tictactoe,500


## Summary
Get most unique rows possible per game. 500 samples each

In [156]:
pd.options.display.max_colwidth = 10000
# setting value of N as 2
N = 500
 
# to column 'Variable'
df_top_n = df2.groupby('game').head(N).reset_index(drop=True)
 
# to column 'Variable'
non_unique = df_top_n >>  group_by(X.game) >> summarize(total =X.game.count()) >> mask(X.total < 500) 
print(non_unique)
samples_per_group_dict = {'first_sealed_auction': 30, 
                          'kuhn_poker': 14,
                          'liars_dice': 80,
                          'nim': 400,
                          'pig': 350}

# sample 80% or so of low prompt occurence samples
#nim also had repeats in test, manually added
top_n_non_unique = df2[df2['game'].isin(non_unique['game']) | df2['game'].isin(['nim'])]

output = top_n_non_unique.groupby('game').apply(lambda group: group.sample(samples_per_group_dict[group.name])).reset_index(drop = True)
#output >>  group_by(X.game) >> summarize(total =X.game.count())

reps = [80 if val == "kuhn_poker" else 70 if val == 'first_sealed_auction' else 10 if val == 'liars_dice' else 2 for val in output.game]
repeat_sample_500 = output.loc[np.repeat(output.index.values, reps)].groupby('game').head(N).reset_index(drop=True)
#repeat_sample_500 >>  group_by(X.game) >> summarize(total =X.game.count())

#get samples from high prompt occurence, combine dfs
df_top_n_highp = df_top_n[~df_top_n['game'].isin(non_unique['game']) & ~df_top_n['game'].isin(['nim'])]

combined = pd.concat([df_top_n,repeat_sample_500], ignore_index=True).reset_index(drop=True).groupby('game').apply(lambda s: s.sample(500)).reset_index(drop=True) 
combined >>  group_by(X.game) >> summarize(total =X.game.count())




                   game  total
2  first_sealed_auction     52
3            kuhn_poker     17
4            liars_dice    140
7                   pig    443


,game,total
0,breakthrough,500
1,connect4,500
2,first_sealed_auction,500
3,kuhn_poker,500
4,liars_dice,500
5,negotiation,500
6,nim,500
7,pig,500
8,python_iterated_prisoners_dilemma,500
9,tictactoe,500


In [160]:
from df2s import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

train_hf = Dataset.from_pandas(pd.DataFrame(combined))
train_hf.push_to_hub("ihughes15234/combined_5000_train")

## Create breakthrough, connect4 training only


In [3]:
con = sqlite3.connect("")

games = ('breakthrough', 'connect4')  # requires to be a tuple

query = f"""
    SELECT * FROM DPO_ttt
    WHERE game IN {games}
"""

df = pd.read_sql_query(query, con)
con.close

<function Connection.close()>

In [4]:
df.shape

(61804, 5)

In [5]:
df2 = df.drop_duplicates()
df2.shape

(50218, 5)

In [6]:
df2 >>  group_by(X.game) >> summarize(total =X.game.count())
df2 = df.drop_duplicates()
df2 >> group_by(X.game) >> summarize(total =X.game.count()) 

,game,total
0,breakthrough,20278
1,connect4,29940


In [7]:
N = 10000
 
# to column 'Variable'
df2 = df2.groupby('game').head(N).reset_index(drop=True)
df2 >>  group_by(X.game) >> summarize(total =X.game.count())

,game,total
0,breakthrough,10000
1,connect4,10000


In [9]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(df2))
train_hf.push_to_hub("ihughes15234/10k_bthrough_connect4_train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.94s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/10k_bthrough_connect4_train/commit/05964c07a8864f05c4663f37feb9eacf06b94451', commit_message='Upload dataset', commit_description='', oid='05964c07a8864f05c4663f37feb9eacf06b94451', pr_url=None, pr_revision=None, pr_num=None)

In [10]:
#pickle data
pd.to_pickle(df2, '/home/GTBench_Thesis/experiments/10k_bthrough_connect4_train.pkl')

## 3000 each game

In [313]:
con = sqlite3.connect("")

query = f"""
    SELECT * FROM DPO_ttt
"""

df = pd.read_sql_query(query, con)
con.close

<function Connection.close()>

In [314]:
df2 = df.drop_duplicates()
df2 >> group_by(X.game) >> summarize(total =X.game.count()) 

,game,total
0,breakthrough,20278
1,connect4,29940
2,first_sealed_auction,53
3,kuhn_poker,17
4,liars_dice,140
5,negotiation,1494
6,nim,560
7,pig,445
8,python_iterated_prisoners_dilemma,662
9,tictactoe,1200


In [315]:
pd.options.display.max_colwidth = 10000
# setting value of N as 2
N = 3000
 
# to column 'Variable'
df_top_n = df2.groupby('game').head(N).reset_index(drop=True)
 
# to column 'Variable'
non_unique = df_top_n >>  group_by(X.game) >> summarize(total =X.game.count()) >> mask(X.total < 3000) 
#print(non_unique)
samples_per_group_dict = {'first_sealed_auction': 40, 
                          'kuhn_poker': 14,
                          'liars_dice': 120,
                          'nim': 530,
                          'pig': 420,
                          'python_iterated_prisoners_dilemma': 630,
                          'negotiation': 1400,
                          'tictactoe': 1200}

# sample 80% or so of low prompt occurence samples
#nim also had repeats in test, manually added
top_n_non_unique = df2[df2['game'].isin(non_unique['game']) | df2['game'].isin(['nim'])]

output = top_n_non_unique.groupby('game').apply(lambda group: group.sample(samples_per_group_dict[group.name])).reset_index(drop = True)
# output >>  group_by(X.game) >> summarize(total =X.game.count())

reps = [300 if val == "kuhn_poker" else 100 if val == 'first_sealed_auction' else 40 if val == 'liars_dice' else 10 for val in output.game]
repeat_sample_3000 = output.loc[np.repeat(output.index.values, reps)].groupby('game').apply(lambda group: group.sample(3000)).reset_index(drop = True)
# repeat_sample_4000 >>  group_by(X.game) >> summarize(total =X.game.count())

# #get samples from high prompt occurence, combine dfs
df_top_n_highp = df_top_n[~df_top_n['game'].isin(non_unique['game'])]
# df_top_n_highp >>  group_by(X.game) >> summarize(total =X.game.count())

combined = pd.concat([df_top_n_highp,repeat_sample_3000], ignore_index=True).reset_index(drop=True).groupby('game').head(N).reset_index(drop=True)

print('Combined Total Entries: \n',combined >>  group_by(X.game) >> summarize(total =X.game.count()))
print('Unique Entries: \n',combined.drop_duplicates() >> group_by(X.game) >> summarize(total =X.game.count()))




Combined Total Entries: 
                                 game  total
0                       breakthrough   3000
1                           connect4   3000
2               first_sealed_auction   3000
3                         kuhn_poker   3000
4                         liars_dice   3000
5                        negotiation   3000
6                                nim   3000
7                                pig   3000
8  python_iterated_prisoners_dilemma   3000
9                          tictactoe   3000
Unique Entries: 
                                 game  total
0                       breakthrough   3000
1                           connect4   3000
2               first_sealed_auction     40
3                         kuhn_poker     14
4                         liars_dice    120
5                        negotiation   1283
6                                nim    530
7                                pig    420
8  python_iterated_prisoners_dilemma    629
9                          ticta

In [311]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(combined))
train_hf.push_to_hub("ihughes15234/3k_each_train")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/3k_each_train/commit/81bb5e87f1d290a1bd2d3f1b6b6f43e31b5af7af', commit_message='Upload dataset', commit_description='', oid='81bb5e87f1d290a1bd2d3f1b6b6f43e31b5af7af', pr_url=None, pr_revision=None, pr_num=None)

## prisoners dilemma

In [17]:
con = sqlite3.connect("")

query = f"""
    SELECT * FROM DPO_ttt
    WHERE game == 'python_iterated_prisoners_dilemma' 
"""

df = pd.read_sql_query(query, con)
con.close

<function Connection.close()>

In [31]:
df2 = df.drop_duplicates()
df2

,sys_prompt,obs_prompt,chosen,rejected,game
0,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Silent>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
1,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Silent>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
2,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Testify>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
3,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Silent>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
5,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Testify>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
...,...,...,...,...,...
3169,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Testify>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
3170,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Silent>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
3171,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Testify>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma
3172,You are a powerful gaming agent who can make p...,You and your partner are in the Prisoner's Dil...,Action:\n<Silent>,"['<Silent>', '<Testify>']",python_iterated_prisoners_dilemma


In [32]:
df2 = df2[["sys_prompt", "obs_prompt", "chosen"]]
df2 = df2.sample(frac=1).reset_index(drop=True)
df2['rejected'] = np.where(df2['chosen']== 'Action:\n<Silent>', 'Action:\n<Testify>', 'Action:\n<Silent>')

In [33]:
df2['prompt'] = "<|system|>\n" + df2['sys_prompt'] + "<|end|>\n" + "<|user|>\n" + df2['obs_prompt'] + "<|end|>\n<|assistant|>\n"

df2['chosen'] = df2['chosen'] + "<|end|>\n<|endoftext|>" 

df2['rejected'] = df2['rejected'] + "<|end|>\n<|endoftext|>" 

In [35]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(df2))
train_hf.push_to_hub("ihughes15234/prisoners_dilemma_dpo_phi")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/prisoners_dilemma_dpo_phi/commit/1d627b247039aaaaae405c2ad46ab0221e8a555c', commit_message='Upload dataset', commit_description='', oid='1d627b247039aaaaae405c2ad46ab0221e8a555c', pr_url=None, pr_revision=None, pr_num=None)

## kuhn poker only

In [3]:
con = sqlite3.connect("")

query = f"""
    SELECT * FROM DPO_kp
    WHERE game == 'kuhn_poker' 
"""

df = pd.read_sql_query(query, con)
con.close

<function Connection.close()>

In [4]:
df2 = df.drop_duplicates()
df2 >> group_by(X.game) >> summarize(total =X.game.count()) 

,game,total
0,kuhn_poker,16


In [5]:
df2 = df2.drop_duplicates('obs_prompt').reset_index()
print(df2['obs_prompt'])

0     Kuhn poker is a simple model zero-sum two-play...
1     Kuhn poker is a simple model zero-sum two-play...
2     Kuhn poker is a simple model zero-sum two-play...
3     Kuhn poker is a simple model zero-sum two-play...
4     Kuhn poker is a simple model zero-sum two-play...
5     Kuhn poker is a simple model zero-sum two-play...
6     Kuhn poker is a simple model zero-sum two-play...
7     Kuhn poker is a simple model zero-sum two-play...
8     Kuhn poker is a simple model zero-sum two-play...
9     Kuhn poker is a simple model zero-sum two-play...
10    Kuhn poker is a simple model zero-sum two-play...
11    Kuhn poker is a simple model zero-sum two-play...
Name: obs_prompt, dtype: object


In [108]:
for i in range(len(df2)):
    print(i)
    print(df2['obs_prompt'][i])

0
Kuhn poker is a simple model zero-sum two-player imperfect-information game, amenable to a complete game-theoretic analysis. In Kuhn poker, the deck includes only three playing cards: a King (K), a Queen (Q), and a Jack (J).
One card is dealt to each player, and the third is put aside unseen. The players take turns either <Bet> to match the bet raised by the opponent or <Pass> to concede the game.
If a player bets, the other player must either call the bet by matching it or fold by conceding the game. If both players pass, the game is over, and the player with the higher-ranking card wins. The card rankings are as follows: King (K) > Queen (Q) > Jack (J).

You are playing Kuhn poker with the opponent. The actions are denoted by <Bet> and <Pass>.
In this match, your card is Queen (Q).

Your legal moves are: <Pass>, <Bet>.
You must choose an legal action to set up advantages.

Your output must be in the following format:

Action:
Your action wrapped with <>, <Pass|Bet> e.g., <Pass>

Pl

In [109]:
custom_rft = ['1', '1pb', '2', '2pb', '0', '0pb', '0p', '0b', '2b', '2p', '1b', '1p']
df2["key"] = ""

In [110]:
for i in range(len(df2)):
    df2['key'][i] = custom_rft[i]
df2

/tmp/ipykernel_106738/3788782994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['key'][i] = custom_rft[i]
/tmp/ipykernel_106738/3788782994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['key'][i] = custom_rft[i]
/tmp/ipykernel_106738/3788782994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['key'][i] = custom_rft[i]
/tmp/ipykernel_106738/3788782994.py:2: SettingWithCopyWarning: 
A value is trying to be set on

,index,sys_prompt,obs_prompt,chosen,rejected,game,key
0,0,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,1
1,1,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,1pb
2,3,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2
3,7,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2pb
4,8,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0
5,9,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0pb
6,75,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0p
7,76,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0b
8,80,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2b
9,81,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2p


In [6]:
cw  = pd.read_csv('')
cw

,Key,Pass,Bet,Key Description
0,0,0.793,0.207,Jack
1,0pb,1.000,0.000,Jack pass oppcall
2,1,1.000,0.000,Queen
3,1pb,0.454,0.546,Queen pass oppcall
4,2,0.379,0.621,King
5,2pb,0.000,1.000,King pass oppcall
6,1p,1.000,0.000,Queen pass
7,1b,0.662,0.338,Queen oppbet
8,2p,0.000,1.000,King opp pass
9,2b,0.000,1.000,King oppbet


In [112]:
join_df = pd.merge(df2, cw, left_on='key', right_on='Key', how='left').drop('Key', axis=1)


,index,sys_prompt,obs_prompt,chosen,rejected,game,key,Pass,Bet,Key Description
0,0,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,1,1.000,0.000,Queen
1,1,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,1pb,0.454,0.546,Queen pass oppcall
2,3,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2,0.379,0.621,King
3,7,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2pb,0.000,1.000,King pass oppcall
4,8,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0,0.793,0.207,Jack
5,9,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0pb,1.000,0.000,Jack pass oppcall
6,75,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0p,0.667,0.333,Jack opppass
7,76,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,"['<Pass>', '<Bet>']",kuhn_poker,0b,1.000,0.000,Jack oppbet
8,80,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2b,0.000,1.000,King oppbet
9,81,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,"['<Pass>', '<Bet>']",kuhn_poker,2p,0.000,1.000,King opp pass


In [125]:
pd.to_pickle(join_df, '')

In [9]:
join_df = pd.read_pickle('')

In [10]:
pass_df = join_df.loc[join_df.index.repeat(join_df['Pass']*100)].assign(chosen= 'Action:\n<Pass>').reset_index(drop=True)
bet_df = join_df.loc[join_df.index.repeat(join_df['Bet']*100)].assign(chosen= 'Action:\n<Bet>').reset_index(drop=True)
comb_df = pd.concat([pass_df, bet_df], sort=False)

In [ ]:
comb_df = comb_df[["sys_prompt", "obs_prompt", "chosen", "Key Description"]]
comb_df = comb_df.sample(frac=1).reset_index(drop=True)


In [12]:
comb_df

,sys_prompt,obs_prompt,chosen,Key Description
0,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,Queen pass
1,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,Queen oppbet
2,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,King oppbet
3,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,Jack pass oppcall
4,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,Queen pass oppcall
...,...,...,...,...
1190,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,Queen
1191,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Bet>,King
1192,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,King
1193,You are a powerful gaming agent who can make p...,Kuhn poker is a simple model zero-sum two-play...,Action:\n<Pass>,Jack


In [116]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(comb_df))
train_hf.push_to_hub("ihughes15234/kp_12000_cfr")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/kp_12000_cfr/commit/a0f18d955e5a8f6d31379cab20b751472523b483', commit_message='Upload dataset', commit_description='', oid='a0f18d955e5a8f6d31379cab20b751472523b483', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
comb_df['rejected'] = np.where(comb_df['chosen']== 'Action:\n<Pass>', 'Action:\n<Bet>', 'Action:\n<Pass>')
#comb_df['rejected'] = np.where(comb_df['chosen']== 'Action:\n<Pass>', 'Action:\n<Bet>', 'Action:\n<I am unsure>')

In [14]:
kp_cfr_drpo = comb_df

In [15]:
## Phi 3.5
kp_cfr_drpo['prompt'] = "<|system|>\n" + kp_cfr_drpo['sys_prompt'] + "<|end|>\n" + "<|user|>\n" + kp_cfr_drpo['obs_prompt'] + "<|end|>\n<|assistant|>\n"

kp_cfr_drpo['chosen'] = kp_cfr_drpo['chosen'] + "<|end|>\n<|endoftext|>" 

kp_cfr_drpo['rejected'] = kp_cfr_drpo['rejected'] + "<|end|>\n<|endoftext|>" 

In [16]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(kp_cfr_drpo))
train_hf.push_to_hub("ihughes15234/kp_cfr_drpo_1200_v2")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/kp_cfr_drpo_1200_v2/commit/c9e952001d49749b19319d452d0100ba8e7f792d', commit_message='Upload dataset', commit_description='', oid='c9e952001d49749b19319d452d0100ba8e7f792d', pr_url=None, pr_revision=None, pr_num=None)

## Tictactoe only

In [2]:
import math
con = sqlite3.connect("")

games = ('tictactoe')  # requires to be a tuple

query = f"""
    SELECT * FROM DPO_ttt_sort
    WHERE game == 'tictactoe' 
"""

df = pd.read_sql_query(query, con)
con.close

<function Connection.close()>

In [3]:
df

,sys_prompt,obs_prompt,chosen,rejected,game
0,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe
1,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C2R2>', '<C3R...",tictactoe
2,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R2>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C3R2>', '<C1R...",tictactoe
3,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C1R3>', '<C2R...",tictactoe
4,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R3>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C1R3>', '<C2R...",tictactoe
...,...,...,...,...,...
25338,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C3R1>', '<C1R2>', '<C3R2>', '<C1R...",tictactoe
25339,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe
25340,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R3>,"['<C1R1>', '<C2R1>', '<C1R2>', '<C3R2>', '<C1R...",tictactoe
25341,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C1R2>', '<C3R2>', '<C2R3>', '<C3R...",tictactoe


In [5]:
df2 = df >> group_by(X.game, X.rejected,X.chosen , X.obs_prompt,X.sys_prompt) >> summarize(total = X.game.count()) 
df2

,sys_prompt,obs_prompt,chosen,rejected,game,total
0,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R2>,"['<C1R1>', '<C1R2>', '<C1R3>', '<C2R3>', '<C3R...",tictactoe,2
1,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R2>,"['<C1R1>', '<C1R2>', '<C1R3>', '<C2R3>', '<C3R...",tictactoe,3
2,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R3>,"['<C1R1>', '<C1R2>', '<C1R3>', '<C2R3>', '<C3R...",tictactoe,28
3,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R3>,"['<C1R1>', '<C1R2>', '<C1R3>', '<C2R3>', '<C3R...",tictactoe,3
4,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C1R2>', '<C1R3>', '<C2R3>']",tictactoe,16
...,...,...,...,...,...,...
1474,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R2>,"['<C3R2>', '<C3R3>']",tictactoe,1
1475,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C3R2>', '<C3R3>']",tictactoe,8
1476,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C3R2>', '<C3R3>']",tictactoe,7
1477,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C3R2>', '<C3R3>']",tictactoe,8


In [6]:
df_sort = df2.sort_values('total',ascending = False).groupby('obs_prompt').head(1)
df_sort.sort_values('total',ascending = False).head(60)

,sys_prompt,obs_prompt,chosen,rejected,game,total
278,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,3180
324,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C3R...",tictactoe,504
342,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C3R...",tictactoe,484
164,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R3>,"['<C1R1>', '<C2R1>', '<C1R2>', '<C3R2>', '<C1R...",tictactoe,462
1068,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C3R2>', '<C1R...",tictactoe,442
322,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R3>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C3R...",tictactoe,300
1034,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C2R1>', '<C3R1>', '<C1R2>', '<C2R2>', '<C3R...",tictactoe,280
132,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C1R2>', '<C2R2>', '<C3R...",tictactoe,276
292,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,269
283,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,260


In [8]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(df2))
train_hf.push_to_hub("ihughes15234/1200_tictactoe")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.49s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/1200_tictactoe/commit/c97151924d176a92dd8f07b9584476f1787bc00d', commit_message='Upload dataset', commit_description='', oid='c97151924d176a92dd8f07b9584476f1787bc00d', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
df_sort_first_only = df_sort.sort_values('total',ascending = False)
df_sort_first_only['max_value'] = 4
df_sort_first_only['repeat1'] = np.ceil(df_sort_first_only['total']/300.0)
df_sort_first_only['repeat_count'] = df_sort_first_only[['repeat1','max_value']].min(axis=1)
df_sort_repeat = df_sort_first_only.loc[df_sort_first_only.index.repeat(df_sort_first_only['repeat_count'])].reset_index(drop=True)
df_sort_repeat['rejected'] = """['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R2>', '<C3R2>', '<C1R3>', '<C2R3>', '<C3R3>']"""


In [9]:
df_sort_repeat

,sys_prompt,obs_prompt,chosen,rejected,game,total,max_value,repeat1,repeat_count
0,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,3180,4,11.0,4.0
1,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,3180,4,11.0,4.0
2,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,3180,4,11.0,4.0
3,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,3180,4,11.0,4.0
4,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,504,4,2.0,2.0
...,...,...,...,...,...,...,...,...,...
1231,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,1,4,1.0,1.0
1232,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R3>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,1,4,1.0,1.0
1233,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C3R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,1,4,1.0,1.0
1234,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C1R1>,"['<C1R1>', '<C2R1>', '<C3R1>', '<C1R2>', '<C2R...",tictactoe,1,4,1.0,1.0


In [10]:
#intitialize list
ttt_list = []

for ind in df_sort_repeat.index:
#get the chosen move and the alternative moves
    alt_move_table = df_sort_repeat.loc[ind][3]
    alt_move = re.findall("\<(.*?)\>", alt_move_table)

    act_move_table = df_sort_repeat.loc[ind][2]
    act_move = re.findall("\<(.*?)\>", act_move_table)[0]
#populate row for each alternative action
    for i in alt_move:
        if len(alt_move) == 1:
            add_to = [df_sort_repeat.loc[ind][0], df_sort_repeat.loc[ind][1], df_sort_repeat.loc[ind][2], "Tic Tac Toe is a two-player game played on a..."]
        else:
            if i == act_move: #no duplicates of chosen action
                continue
            add_to = [df_sort_repeat.loc[ind][0], df_sort_repeat.loc[ind][1], df_sort_repeat.loc[ind][2], "Action:\n<" + i + ">"]
        ttt_list.append(add_to)

ttt_train = pd.DataFrame(ttt_list)

#Check duplicates        
print(len(ttt_list))
print(ttt_train.shape[0])
ttt_train.columns = ['sys_prompt', 'obs_prompt', 'chosen', 'rejected']
print(ttt_train.head(1))


9888
9888
                                          sys_prompt  \
0  You are a powerful gaming agent who can make p...   

                                          obs_prompt           chosen  \
0  Tic Tac Toe is a two-player game played on a g...  Action:\n<C2R2>   

          rejected  
0  Action:\n<C1R1>  


In [57]:
ttt_train

,sys_prompt,obs_prompt,chosen,rejected
0,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C1R1>
1,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C2R1>
2,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C3R1>
3,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C1R2>
4,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C3R2>
...,...,...,...,...
9067,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C1R2>
9068,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C3R2>
9069,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C1R3>
9070,You are a powerful gaming agent who can make p...,Tic Tac Toe is a two-player game played on a g...,Action:\n<C2R2>,Action:\n<C2R3>


In [37]:
#intitialize list
ttt_list = []

for ind in df_sort.index:
#get the chosen move and the alternative moves
    alt_move_table = df_sort.loc[ind][3]
    alt_move = re.findall("\<(.*?)\>", alt_move_table)

    act_move_table = df_sort.loc[ind][2]
    act_move = re.findall("\<(.*?)\>", act_move_table)[0]
#populate row for each alternative action
    for i in alt_move:
        if len(alt_move) == 1:
            add_to = [df_sort.loc[ind][0], df_sort.loc[ind][1], df_sort.loc[ind][2], "Tic Tac Toe is a two-player game played on a..."]
        else:
            if i == act_move: #no duplicates of chosen action
                continue
            add_to = [df_sort.loc[ind][0], df_sort.loc[ind][1], df_sort.loc[ind][2], "Action:\n<" + i + ">"]
        ttt_list.append(add_to)

ttt_train = pd.DataFrame(ttt_list).drop_duplicates()

#Check duplicates        
print(len(ttt_list))
print(ttt_train.shape[0])
ttt_train.columns = ['sys_prompt', 'obs_prompt', 'chosen', 'rejected']
print(ttt_train.head(1))


3462
3462
                                          sys_prompt  \
0  You are a powerful gaming agent who can make p...   

                                          obs_prompt           chosen  \
0  Tic Tac Toe is a two-player game played on a g...  Action:\n<C2R2>   

          rejected  
0  Action:\n<C1R1>  


In [13]:
ttt_train = ttt_train.sample(frac=1).reset_index(drop=True)


In [15]:
val = ttt_train['obs_prompt'][0]
ttt_train[ttt_train['obs_prompt'] == val].iloc[0]

sys_prompt    You are a powerful gaming agent who can make p...
obs_prompt    Tic Tac Toe is a two-player game played on a g...
chosen                    Action:\n<C1R2><|end|>\n<|endoftext|>
rejected                  Action:\n<C3R1><|end|>\n<|endoftext|>
prompt        <|system|>\nYou are a powerful gaming agent wh...
Name: 0, dtype: object

In [58]:
## Llama
ttt_train['prompt'] = "<|begin_of_text|><|start_header_id|>system<|end_header_id|> \n" + ttt_train['sys_prompt'] + "<|eot_id|>" + "<|start_header_id|>user<|end_header_id|> \n" + ttt_train['obs_prompt'] + "<|eot_id|> <|start_header_id|>assistant<|end_header_id|>"

ttt_train['chosen'] = ttt_train['chosen'] + "<|eot_id|>" 

ttt_train['rejected'] = ttt_train['rejected'] + "<|eot_id|>" 

In [11]:
## Phi 3.5
ttt_train['prompt'] = "<|system|>\n" + ttt_train['sys_prompt'] + "<|end|>\n" + "<|user|>\n" + ttt_train['obs_prompt'] + "<|end|>\n<|assistant|>\n"

ttt_train['chosen'] = ttt_train['chosen'] + "<|end|>\n<|endoftext|>" 

ttt_train['rejected'] = ttt_train['rejected'] + "<|end|>\n<|endoftext|>" 

In [14]:
from datasets import Dataset

train_hf = Dataset.from_pandas(pd.DataFrame(ttt_train))
train_hf.push_to_hub("ihughes15234/ttt_dpo_phi_v2_all_other")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/ttt_dpo_phi_v2_all_other/commit/ddf89a4f9c7971a7a1c93d8f04c613f6cbc147bd', commit_message='Upload dataset', commit_description='', oid='ddf89a4f9c7971a7a1c93d8f04c613f6cbc147bd', pr_url=None, pr_revision=None, pr_num=None)

## DPO Build script

In [323]:
#intitialize list
ttt_list = []

for ind in df.index:
#get the chosen move and the alternative moves
    alt_move_table = df.loc[ind][3]
    alt_move = re.findall("\<(.*?)\>", alt_move_table)

    act_move_table = df.loc[ind][2]
    act_move = re.findall("\<(.*?)\>", act_move_table)[0]
#populate row for each alternative action
    for i in alt_move:
        if i == act_move: #no duplicates of chosen action
            continue
        add_to = [df.loc[ind][0], df.loc[ind][1], df.loc[ind][2], "Action:\n<" + i + ">"]
        ttt_list.append(add_to)

ttt_train = pd.DataFrame(ttt_list).drop_duplicates()

#Check duplicates        
print(len(ttt_list))
print(ttt_train.shape[0])
ttt_train.columns = ['sys_prompt', 'obs_prompt', 'chosen', 'rejected']
print(ttt_train.head(1))



1716
676
                                                                                            sys_prompt  \
0  You are a powerful gaming agent who can make proper decisions to beat the user in gaming tasks. ...   

                                                                                            obs_prompt  \
0  Tic Tac Toe is a two-player game played on a grid. Players take turns marking a space with their...   

            chosen         rejected  
0  Action:\n<C2R2>  Action:\n<C1R1>  


In [193]:
pd.options.display.max_colwidth = 100

In [285]:
#ttt_train['messages'] = ttt_train[0].str[:-1] + ', {"role": "assistant", "content": ' + ttt_train[1] + '"}]'
#ttt_train['messages'][0]

In [335]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

#For SFT, only need the chosen not the rejected answers
train, test = train_test_split(ttt_train[['sys_prompt', 'obs_prompt', 'chosen']].drop_duplicates(), test_size=0.1)

train_hf = Dataset.from_pandas(pd.DataFrame(train))
test_hf = Dataset.from_pandas(pd.DataFrame(test))

ds = DatasetDict()

ds['train'] = train_hf
ds['test'] = test_hf

print(ds)


DatasetDict({
    train: Dataset({
        features: ['sys_prompt', 'obs_prompt', 'chosen', '__index_level_0__'],
        num_rows: 150
    })
    test: Dataset({
        features: ['sys_prompt', 'obs_prompt', 'chosen', '__index_level_0__'],
        num_rows: 17
    })
})


In [ ]:
def construct_init_messages(system_prompt, user_prompt, chosen):
        msgs = [
            {
                'role': 'system',
                'content': system_prompt
            },
            {
                'role': 'user',
                'content': user_prompt
            },
            {
                'role': 'assistant',
                'content': chosen
            }
        ]
        return msgs

In [325]:
ds.push_to_hub("ihughes15234/tictactoe_sft")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/ihughes15234/tictactoe_sft/commit/5ba19ab70ee1d38706e5f208b17b5bf2499c8680', commit_message='Upload dataset', commit_description='', oid='5ba19ab70ee1d38706e5f208b17b5bf2499c8680', pr_url=None, pr_revision=None, pr_num=None)

In [376]:
def constr_msgs(examples):
        system_prompt = examples["sys_prompt"]
        user_prompt       = examples["obs_prompt"]
        chosen      = examples["chosen"]
        EOS_TOKEN = ''#tokenizer.eos_token # Must add EOS_TOKEN
        texts = []
        for system_prompt, user_prompt, chosen in zip(system_prompt, user_prompt, chosen):
            # Must add EOS_TOKEN, otherwise your generation will go on forever!
            msgs = [
            {
                'role': 'system',
                'content': system_prompt
            },
            {
                'role': 'user',
                'content': user_prompt
            },
            {
                'role': 'assistant',
                'content': chosen
            }
            ]
            text = msgs
            texts.append(text)
        return { "text" : texts, }

dataset2 = ds.map(constr_msgs, batched = True,)

Map: 100%|██████████| 150/150 [00:00<00:00, 8189.01 examples/s]


In [369]:
ds

DatasetDict({
    train: Dataset({
        features: ['sys_prompt', 'obs_prompt', 'chosen', '__index_level_0__'],
        num_rows: 150
    })
    test: Dataset({
        features: ['sys_prompt', 'obs_prompt', 'chosen', '__index_level_0__'],
        num_rows: 17
    })
})

In [5]:
from datasets import Dataset
from datasets import load_dataset

In [14]:
ds = load_dataset("ihughes15234/ttt_dpo_phi_v2_all_other")

Generating train split: 100%|██████████| 9888/9888 [00:00<00:00, 70601.83 examples/s]


In [49]:
training_df1 = pd.DataFrame()
training_df1['obs_prompt'] = ds['train']['obs_prompt']
training_df = training_df1.drop_duplicates().reset_index(drop=True)
training_df.shape

(1229, 1)

In [50]:
import json
import pandas as pd

game_list = ['tictactoe']
# training_df = pd.read_pickle('/home/GTBench_Thesis/experiments/3k_each_train.pkl')

# training_df['obs_prompt']


for g in game_list:
    print('\n', g, '\n')
    fp = '/home/GTBench_Thesis/GTBench/experiments/phi_3_tictactoe_dpo_5epoch_v7/phi_3_mini/' + g +'/prompt_agent_phi_3_mini_random_agent_phi_3_mini.jsonl'

    with open(fp, 'r') as json_file:
        json_list = list(json_file)
    result = json.loads(json_list[0])
    agent_moves = 0
    prompt_in_training = 0
    for json_str in json_list:
        
        result = json.loads(json_str)
        
        if result['matches'][0]['status'] == "Abnormal":
            continue
        
        total_steps = len(result['matches'][0]['steps'])
        for i in range(0, total_steps):
            if result['matches'][0]['steps'][i]['agent'] == 'PromptAgent':
                agent_moves += 1
                if result['matches'][0]['steps'][i]['queries'] == []:
                    continue
                prompt_check = result['matches'][0]['steps'][i]['queries'][0]['messages'][1]['content'] 
                if prompt_check in training_df['obs_prompt'].values:
                    prompt_in_training += 1
                    
    print('Total Moves: ', agent_moves)
    print('Prompt in Training: ', prompt_in_training)
    if agent_moves == 0:
        print("no agent moves")
    else:
        print( 'Percent in Training', prompt_in_training/agent_moves * 100, '%')


 tictactoe 

Total Moves:  354
Prompt in Training:  100
Percent in Training 28.24858757062147 %


In [51]:
ds = load_dataset("ihughes15234/prisoners_dilemma_dpo_phi")

Generating train split: 100%|██████████| 855/855 [00:00<00:00, 44397.63 examples/s]


In [52]:
training_df1 = pd.DataFrame()
training_df1['obs_prompt'] = ds['train']['obs_prompt']
training_df = training_df1.drop_duplicates().reset_index(drop=True)
training_df.shape

(855, 1)

In [53]:
import json
import pandas as pd

game_list = ['prisoners_dilemma']
# training_df = pd.read_pickle('/home/GTBench_Thesis/experiments/3k_each_train.pkl')

# training_df['obs_prompt']

for g in game_list:
    print('\n', g, '\n')
    fp = '/home/GTBench_Thesis/GTBench/experiments/phi35_pd_dpo10epoch_1200/phi_3_mini/' + g +'/prompt_agent_phi_3_mini_random_agent_phi_3_mini.jsonl'

    with open(fp, 'r') as json_file:
        json_list = list(json_file)
    result = json.loads(json_list[0])
    agent_moves = 0
    prompt_in_training = 0
    for json_str in json_list:
        
        result = json.loads(json_str)
        
        if result['matches'][0]['status'] == "Abnormal":
            continue
        
        total_steps = len(result['matches'][0]['steps'])
        for i in range(0, total_steps):
            if result['matches'][0]['steps'][i]['agent'] == 'PromptAgent':
                agent_moves += 1
                if result['matches'][0]['steps'][i]['queries'] == []:
                    continue
                prompt_check = result['matches'][0]['steps'][i]['queries'][0]['messages'][1]['content'] 
                if prompt_check in training_df['obs_prompt'].values:
                    prompt_in_training += 1
                    
    print('Total Moves: ', agent_moves)
    print('Prompt in Training: ', prompt_in_training)
    if agent_moves == 0:
        print("no agent moves")
    else:
        print( 'Percent in Training', prompt_in_training/agent_moves * 100, '%')


 prisoners_dilemma 

Total Moves:  326
Prompt in Training:  229
Percent in Training 70.24539877300614 %
